In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [2]:
###input
ymdb_output_final = '../../../Data/database/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter_add_no_ec.csv'

target_smiles_path = '../../../Results/lipid/target_smiles_lipid.pickle'
target_model_smiles_path = '../../../Results/lipid/yeast_met_lipid.pickle'
yeast8_metabolites_path = '../../../Data/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'

cut_off = 0.3
num = 'all'
num_process = 5
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re/'
sce_gene_clean_ec = '../../../Results/ECnumber_prediction/Saccharomyces_cerevisiae_teacher_maxsep.csv'
DeepEC_path = '../../../Results/ECnumber_prediction/Sce_DeepECv2.txt'
sce_uniprot_path = '../../../Data/database/uniprot/uniprotkb_organism_id_559292_2023_11_08.tsv'

yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/miss_met_id.json'
rxndb_to_model_with_target_success_path = cut_off_path + 'rxndb_to_model_with_target_topall_0.3_success.csv'
metabolites_info_to_GEM_path_not_lipid_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_top50_0.3.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'


###output

rxndb_to_model_add_ec_path = cut_off_path + f'rxndb_to_model_add_ec_top{num}_{cut_off}.csv'
rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}.csv'
rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}.csv'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'



In [3]:
def process_smiles_annotation(i,yeast_total_smiles):    ###The highest similarity scores of all the reactants and model metabolites of rxn were obtained
    result = {'smile': i, 'score': 0, 'sim_smile': ''}
    score_init = 0
    sim_smile = ' '
    for j in yeast_total_smiles:
        # tmp_score = compare_smiles(i, j)  
        if compare_smiles_inchikey(i,j) :
            score_init = 1
            sim_smile = j
            break
    result['score'] = score_init
    result['sim_smile'] = sim_smile
    return result
def calculate_smile_max_score_annotation(rxndb_all_reactant_smile,yeast_total_smiles,num_process=30):
    smile_max_score = {'smile': [], 'score': [], 'sim_smile': []}

    pool = multiprocessing.Pool(num_process)

    partial_process_smiles = partial(process_smiles_annotation,yeast_total_smiles=yeast_total_smiles)
    results = list(tqdm(pool.imap(partial_process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))
    # results = list(tqdm(pool.imap(process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))

    pool.close()
    pool.join()

    for result in results:
        smile_max_score['smile'].append(result['smile'])
        smile_max_score['score'].append(result['score'])
        smile_max_score['sim_smile'].append(result['sim_smile'])
        
    return(smile_max_score)

In [4]:
#run find_no_ec_reaction_lipid_re.ipynb to get the rxndb_to_model_with_target_topall_0.3_success.csv file
data_add_no_ec = pd.read_csv(rxndb_to_model_with_target_success_path)
data_no_ec = data_add_no_ec[data_add_no_ec['EC number'].isnull()]
data_ec = data_add_no_ec[~data_add_no_ec['EC number'].isnull()]

In [5]:
data = pd.read_csv(rxndb_to_model_path)
print('data:', data.shape)

print('Unique NO values in data before merge:', data['NO'].nunique())

data = pd.concat([data, data_ec])


data = data.drop(columns=['NO'])

data = data.reset_index(drop=True)
data.index.name = 'NO'
data = data.reset_index()

print( data['NO'].nunique())
data.to_csv(rxndb_to_model_add_ec_path,index=None)


data: (854584, 5)
Unique NO values in data before merge: 854584
966965


In [6]:
model = cobra.io.load_yaml_model(yeast870_path)

model_genes = model.genes

model_genes = [x.id for x in model_genes]

### CLEAN

In [7]:
clean_gene2ec_dict = get_gene2ec_dict_clean(sce_gene_clean_ec)

### DeepECv2

In [8]:
DeepEC_gene2ec_dict = get_gene2ec_dict_DeepEC(DeepEC_path)

In [9]:
clean_deepec_gene2ec_dict = {}
for k,v in clean_gene2ec_dict.items():
    if k in DeepEC_gene2ec_dict or k in model_genes:
        clean_deepec_gene2ec_dict[k] = v
clean_deepec_ec2gene_dict = get_ec2gene_dict_clean(clean_deepec_gene2ec_dict)
print(len(clean_deepec_ec2gene_dict))

161


In [10]:
rxndb_with_GPR = rxndb_gene_annotation(rxndb_to_model_add_ec_path,clean_deepec_ec2gene_dict)
rxndb_with_GPR = rxndb_with_GPR[rxndb_with_GPR['GENE'].apply(lambda x: len(x) > 0)]
data_no_ec['GENE'] = None
data_no_ec['GENE'] = data_no_ec['GENE'].apply(lambda x: [])
data_no_ec['EC number'] = data_no_ec['EC number'].apply(lambda x: [])
print('rxndb with GPR:',rxndb_with_GPR.shape)


rxndb all ec num 112
rxndb with GPR: (966446, 6)


/tmp/ipykernel_2294769/3407858261.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_ec['GENE'] = None
/tmp/ipykernel_2294769/3407858261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_ec['GENE'] = data_no_ec['GENE'].apply(lambda x: [])
/tmp/ipykernel_2294769/3407858261.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [11]:
print('rxndb_with_GPR:',rxndb_with_GPR.shape)

rxndb_with_GPR = pd.concat([rxndb_with_GPR,data_no_ec])
rxndb_with_GPR = rxndb_with_GPR.drop(columns=['NO'])
rxndb_with_GPR = rxndb_with_GPR.reset_index(drop=True)
rxndb_with_GPR.index.name = 'NO'
rxndb_with_GPR = rxndb_with_GPR.reset_index()

print( rxndb_with_GPR['NO'].nunique())
# rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)

rxndb_with_GPR: (966446, 6)
1022175


In [12]:
rxndb_with_GPR['NO'] = rxndb_with_GPR['NO'].apply(lambda x: f'rxnl{x}')
rxndb_with_GPR.head()


,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final,GENE
0,rxnl0,[1.1.1],MNXR100471_reverse,O=C([O-])C[C@@H](O)C(=O)[O-]>>O=C([O-])CC(=O)C...,O=C([O-])C[C@@H](O)C(=O)[O-].NC(=O)c1ccc[n+]([...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
1,rxnl1,[1.1.1],MNXR100471_reverse,C[C@H](O)CC(=O)[O-]>>CC(=O)CC(=O)[O-],C[C@H](O)CC(=O)[O-].NC(=O)c1ccc[n+]([C@@H]2O[C...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
2,rxnl2,[1.1.1],MNXR100471_reverse,CC(C)C(C(=O)[O-])[C@@H](O)C(=O)[O-]>>CC(C)C(C(...,CC(C)C(C(=O)[O-])[C@@H](O)C(=O)[O-].NC(=O)c1cc...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
3,rxnl3,[1.1.1],MNXR100471_reverse,C[C@H](O)C([NH3+])C(=O)[O-]>>CC(=O)C([NH3+])C(...,C[C@H](O)C([NH3+])C(=O)[O-].NC(=O)c1ccc[n+]([C...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
4,rxnl4,[3.1.3],MNXR114735,O=P([O-])([O-])O.OCC1OC(O)C(O)C(O)C1O>>O=P([O-...,O=P([O-])([O-])O.OCC1OC(O)C(O)C(O)C1O>>O=P([O-...,"[YDR063W, YDR051C, YKR053C, YIL002C, YBR022W, ..."


In [13]:
rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)


### rxndb metabolite annotation

In [14]:
#rxndb all metabolites
rxndb_all_smile = get_all_met_smile(rxndb_GPR_to_model_path)


100%|██████████| 1022175/1022175 [09:51<00:00, 1728.50it/s]

41985


In [15]:
#metabolite comparison
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
smile_max_score = calculate_smile_max_score_annotation(rxndb_all_smile,yeast8_total_smiles,num_process=num_process)
smile_max_score_pd = pd.DataFrame(smile_max_score)
smile_max_score_pd.to_csv(rxndb_met_max_score_met_annotation_file,index=None)


100%|██████████| 41985/41985 [18:24<00:00, 38.02it/s]


In [16]:
#assign ID
yeast8_id_smiles_mapping = get_yeast8_id_smiles_mapping(yeast8_metabolites_path)
smile_max_score_pd_with_ID = assign_ID_to_RXNDB_met(smile_max_score_pd,yeast8_id_smiles_mapping,s = 'lipid')
metabolites_info_to_GEM = get_new_met_smile_list(smile_max_score_pd_with_ID)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)



/home/haohao/underground/Yeast-MetaTwin/Code/retrosynthesis/lipid_retrosynthesis/../common.py:1397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['smiles'] = yeast_id_mapping['smiles'].apply(normalize_smiles)
/home/haohao/underground/Yeast-MetaTwin/Code/retrosynthesis/lipid_retrosynthesis/../common.py:1398: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['inchikey0'] = yeast_id_mapping['smiles'].apply(smiles2inchikey0)
100%|██████████| 41985/41985 [00:45<00:00, 915.22it/s]


In [19]:
metabolites_info_to_GEM_path_not_lipid = pd.read_csv(metabolites_info_to_GEM_path_not_lipid_path)
metabolites_info_to_GEM_path_not_lipid['inchikey0'] = metabolites_info_to_GEM_path_not_lipid['new_met_smiles'].apply(smiles2inchikey0)
for i in tqdm(range(metabolites_info_to_GEM.shape[0]),total=metabolites_info_to_GEM.shape[0]):
    if 'sl_' not in metabolites_info_to_GEM['ID'][i]:
        continue
    if smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i]) in metabolites_info_to_GEM_path_not_lipid['inchikey0'].values:
        metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)

  0%|          | 0/41985 [00:00<?, ?it/s]/tmp/ipykernel_2294769/1497864285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
/tmp/ipykernel_2294769/1497864285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].value

### Generate equation according to metabolite ID

In [21]:
smiles_to_id_mapping = get_smiles_to_id_mapping(metabolites_info_to_GEM_path)
print(len(smiles_to_id_mapping))

41985


In [22]:
rxndb_total_info_to_model = get_rxndb_to_model_total_info(rxndb_GPR_to_model_path,smiles_to_id_mapping)
rxndb_total_info_to_model = rxndb_total_info_to_model[['NO','EC_number','templateID','rxn_smiles_basic','rxn_smiles_final','GPR','equation_dict','equation']]
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

(733334, 8)


### check the formula of the model metabolites in the predicted reaction

In [23]:
metabolites_info_to_GEM,rxndb_total_info_to_model = check_met_and_reaction_formula(yeast8_metabolites_path,metabolites_info_to_GEM_path,rxndb_total_info_to_model_path)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

formula error_met set()


0it [00:00, ?it/s]


(733334, 8)


### remove duplicate reactions and existing reactions

In [24]:
same_rxndb_reaction = get_same_rxndb_reaction_mapping(rxndb_total_info_to_model_path)
rxndb_total_info_to_model = get_no_same_rxndb_reaction_parallelize(rxndb_total_info_to_model_path,get_no_same_rxndb_reaction,same_rxndb_reaction,n_cores=num_process)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

(733334, 8)


 59%|█████▊    | 430764/733334 [00:17<00:12, 24218.14it/s]

100%|██████████| 733334/733334 [00:28<00:00, 25775.15it/s]


(87896, 8)


In [25]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
rxndb_to_model_total_info = pd.read_csv(rxndb_total_info_to_model_path)


### Find out the reactions that already exist in the model

In [26]:
with open(miss_met_id_file) as f:
    miss_met_id = json.load(f) 
model = cobra.io.load_yaml_model(yeast870_path)


### yeast8_smiles_list, ID with smiles

In [27]:
model_excel, metID_with_smiles, yeast8_smiles_lst = process_model(yeast8_metabolites_path, miss_met_id)


2124
['s_0006', 's_0007', 's_0008', 's_0009', 's_0010', 's_0011', 's_0012', 's_0013', 's_0014', 's_0015', 's_0016', 's_0017', 's_0018', 's_0019', 's_0020', 's_0021', 's_0022', 's_0023', 's_0024', 's_0025', 's_0026', 's_0027', 's_0028', 's_0029', 's_0030', 's_0031', 's_0032', 's_0033', 's_0034', 's_0035', 's_0036', 's_0037', 's_0038', 's_0039', 's_0042', 's_0045', 's_0048', 's_0051', 's_0054', 's_0056', 's_0058', 's_0060', 's_0061', 's_0062', 's_0063', 's_0064', 's_0065', 's_0066', 's_0067', 's_0068', 's_0069', 's_0075', 's_0076', 's_0077', 's_0078', 's_0079', 's_0080', 's_0086', 's_0087', 's_0116', 's_0117', 's_0118', 's_0119', 's_0120', 's_0121', 's_0122', 's_0123', 's_0124', 's_0125', 's_0126', 's_0127', 's_0129', 's_0131', 's_0132', 's_0133', 's_0134', 's_0135', 's_0136', 's_0137', 's_0138', 's_0139', 's_0140', 's_0141', 's_0142', 's_0143', 's_0144', 's_0145', 's_0146', 's_0147', 's_0148', 's_0149', 's_0150', 's_0152', 's_0153', 's_0154', 's_0155', 's_0156', 's_0157', 's_0158', 's_0

In [28]:
rxndb_met_with_s_smiles_lst = get_rxndb_met_with_s_smiles_lst(rxndb_total_info_to_model_path,model_excel,miss_met_id)

reaID_with_smiles_dict = get_reaID_with_smiles_dict(model, model_excel, miss_met_id,rxndb_met_with_s_smiles_lst,metID_with_smiles)


(87896, 8)


 45%|████▌     | 39968/87896 [00:12<00:41, 1144.92it/s] 

100%|██████████| 87896/87896 [00:19<00:00, 4538.38it/s] 


842
['Nc1nc2c(ncn2C2OC(CO)C(O)C2O)c(=O)[nH]1', 'Nc1nc2c(ncn2C2OC(CO)C(O)C2OP(=O)(O)O)c(=O)[nH]1', 'Nc1nc2c(ncn2C2OC(CO)C(OP(=O)(O)O)C2O)c(=O)[nH]1', 'Nc1ncnc2c1ncn2C1OC(COP(=O)(O)O)C(OP(=O)(O)O)C1O', 'Nc1ncnc2c1ncn2C1OC(CO)C(OP(=O)(O)O)C1O', 'Nc1ncnc2c1ncn2C1OC(CO)C(O)C1O', 'Nc1ncnc2c1ncn2C1OC(CO)C(O)C1OP(=O)(O)O', 'CCC(=O)C(=O)[O-]', 'NC(CCO)C(=O)O', 'CC=O']
2702
1381
{'r_0003': ['CC(O)C(C)O', 'CC(=O)C(C)O'], 'r_0012': ['NC(CCC(=O)O)C(=O)O', 'O=C(O)C1CCC=N1'], 'r_0013': ['CSCCC(=O)C(=O)O', 'CSCCC(=O)C(=O)COP(=O)(O)O', 'O=C[O-]'], 'r_0014': ['Nc1c(NCC(O)C(O)C(O)COP(=O)(O)O)[nH]c(=O)[nH]c1=O', 'Nc1nc(NCC(O)C(O)C(O)COP(=O)(O)O)c(N)c(=O)[nH]1'], 'r_0015': ['Nc1nc(NCC(O)C(O)C(O)COP(=O)(O)O)c(N)c(=O)[nH]1', 'Nc1nc(NC2OC(COP(=O)(O)O)C(O)C2O)c(N)c(=O)[nH]1'], 'r_0016': ['CCC(=O)C(=O)[O-]', 'CC(=O)C(=O)[O-]', 'CCC(O)(C(C)=O)C(=O)[O-]', 'O=C=O'], 'r_0017': ['Nc1nc2c(c(=O)[nH]1)N=C(COP(=O)(O)OP(=O)(O)O)CN2', 'Nc1nc2c(c(=O)[nH]1)N=C(CO)CN2'], 'r_0018': ['NC(CCCC(=O)O)C(=O)O', 'NC(CCC(=O)O)C(=O)O'

In [29]:
yeast8_reaction_in_rxndb = get_yeast8_reaction_in_rxndb_parallel(rxndb_total_info_to_model_path,reaID_with_smiles_dict,model_excel,miss_met_id,num_process=num_process)


(87896, 8)
(7192, 8)


In [30]:
with open(yeast8_reaction_in_rxndb_json, 'w') as json_file:
    json.dump(yeast8_reaction_in_rxndb, json_file,indent=4)

In [31]:
rxndb_to_model_total_info['equation_dict'] = rxndb_to_model_total_info['equation_dict'].apply(lambda x:literal_eval(x))
print(rxndb_to_model_total_info.shape)
# rxndb_to_model_total_info['equation_dict'] = rxndb_to_model_total_info['equation_dict'].apply(lambda x:{k:v for k, v in x.items() if k not in miss_met_id})
# rxndb_to_model_total_info = rxndb_to_model_total_icnfo[~rxndb_to_model_total_info['equation_dict'].apply(lambda x : all('s_' in k for k, v in x.items()))]
rxndb_to_model_total_info = rxndb_to_model_total_info[rxndb_to_model_total_info['equation_dict'].apply(lambda x : any('sl_' in k for k, v in x.items()))]
print(rxndb_to_model_total_info.shape)
# rxndb_to_model_total_info = rxndb_to_model_total_info[~rxndb_to_model_total_info['equation_dict'].apply(lambda x : all('sn_' in k for k, v in x.items()))]
# print(rxndb_to_model_total_info.shape)

(87896, 8)
(77862, 8)


In [32]:
rxndb_to_model_total_info.to_csv(rxndb_total_info_to_model_path,index=None)
